In [1]:
import pickle
#import os

#DATADIR = "data"
    
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

2022-05-05 16:55:33.148364: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-05 16:55:33.177619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 16:55:33.209429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 16:55:33.209631: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape= X.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1,  activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 63, 63, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 64)                8

2022-05-05 16:55:39.832291: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 16:55:39.832531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 16:55:39.832684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 16:55:39.833001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 16:55:39.833123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
 
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
   
    print(e)

1 Physical GPUs, 1 Logical GPU


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='loss', patience=3) 

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32,epochs=10, validation_split=0.3, callbacks=[callback])


Epoch 1/10


2022-05-05 16:55:50.855391: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-05-05 16:55:51.022227: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


166/166 [==============================] - 7s 34ms/step - loss: 1.9635 - accuracy: 0.6455 - val_loss: 0.5941 - val_accuracy: 0.6915
Epoch 2/10
166/166 [==============================] - 5s 32ms/step - loss: 0.5159 - accuracy: 0.7433 - val_loss: 0.5087 - val_accuracy: 0.7564
Epoch 3/10
166/166 [==============================] - 6s 34ms/step - loss: 0.4320 - accuracy: 0.7935 - val_loss: 0.4923 - val_accuracy: 0.7732
Epoch 4/10
166/166 [==============================] - 5s 32ms/step - loss: 0.3484 - accuracy: 0.8440 - val_loss: 0.5064 - val_accuracy: 0.7833
Epoch 5/10
166/166 [==============================] - 5s 32ms/step - loss: 0.2651 - accuracy: 0.8931 - val_loss: 0.5738 - val_accuracy: 0.7723
Epoch 6/10
166/166 [==============================] - 5s 33ms/step - loss: 0.2965 - accuracy: 0.8672 - val_loss: 0.5904 - val_accuracy: 0.7749
Epoch 7/10
166/166 [==============================] - 5s 33ms/step - loss: 0.1978 - accuracy: 0.9223 - val_loss: 0.6175 - val_accuracy: 0.7838
Epoch 8/10

In [6]:
model.save('CNN_64X3.h5')

In [1]:
from numba import cuda
cuda.select_device(0)
cuda.close()